# Ecobici Datos 

In [1]:
#Crear la sesión en Spark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("EcoBiciTarea").getOrCreate()


24/10/20 12:57:52 WARN Utils: Your hostname, Shie-Hassaikai resolves to a loopback address: 127.0.1.1; using 192.168.100.17 instead (on interface wlp1s0)
24/10/20 12:57:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/20 12:57:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#Crear el df
csv_directory = "/home/atenea/Macrodatos/Tarea Ecobici"
df = spark.read.csv(f"{csv_directory}/*.csv", header=True, inferSchema=True)


In [3]:
df.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: integer (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- Fecha_Retiro: date (nullable = true)
 |-- Hora_Retiro: timestamp (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- Fecha_Arribo: date (nullable = true)
 |-- Hora_Arribo: timestamp (nullable = true)



In [4]:
df.show(10)

+--------------+------------+----+---------------------+------------+--------------------+---------------------+------------+--------------------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|         Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|         Hora_Arribo|
+--------------+------------+----+---------------------+------------+--------------------+---------------------+------------+--------------------+
|             M|          28|2401|                  118|  2012-12-01|2024-10-20 00:00:...|                  118|  2012-12-01| 2024-10-20 00:15:00|
|             M|          26|1540|                   73|  2012-12-01|2024-10-20 00:00:...|                   63|  2012-12-01|2024-10-20 00:06:...|
|             F|          30|2197|                   64|  2012-12-01|2024-10-20 00:00:...|                  167|  2012-12-01| 2024-10-20 00:16:21|
|             M|          27|1421|                   63|  2012-12-01|2024-10-20 00:00:...|                   53|  2012

## Añadir columnas para Fecha y Hora de retiro

In [5]:
from pyspark.sql import functions as F
# Combinar Fecha y Hora de Retiro
df_Ret = df.withColumn("fecha_hora_retiro", F.concat_ws(" ", "Fecha_Retiro", F.date_format("Hora_Retiro", "HH:mm:ss"))) \
            .withColumn("fecha_hora_retiro", F.to_timestamp("fecha_hora_retiro"))

# Extraer año, mes, día, minuto y segundo
df_Ret = df_Ret.withColumn("anio_retiro", F.year("fecha_hora_retiro")) \
                .withColumn("mes_retiro", F.month("fecha_hora_retiro")) \
                .withColumn("dia_retiro", F.dayofmonth("fecha_hora_retiro")) \
                .withColumn("minuto_retiro", F.minute("fecha_hora_retiro")) \
                .withColumn("segundo_retiro", F.second("fecha_hora_retiro"))

df_Ret.show(10)

+--------------+------------+----+---------------------+------------+--------------------+---------------------+------------+--------------------+-------------------+-----------+----------+----------+-------------+--------------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|         Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|         Hora_Arribo|  fecha_hora_retiro|anio_retiro|mes_retiro|dia_retiro|minuto_retiro|segundo_retiro|
+--------------+------------+----+---------------------+------------+--------------------+---------------------+------------+--------------------+-------------------+-----------+----------+----------+-------------+--------------+
|             M|          28|2401|                  118|  2012-12-01|2024-10-20 00:00:...|                  118|  2012-12-01| 2024-10-20 00:15:00|2012-12-01 00:00:14|       2012|        12|         1|            0|            14|
|             M|          26|1540|                   73|  2012-12-01|2024-10-20 

In [6]:
df_Ret.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: integer (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- Fecha_Retiro: date (nullable = true)
 |-- Hora_Retiro: timestamp (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- Fecha_Arribo: date (nullable = true)
 |-- Hora_Arribo: timestamp (nullable = true)
 |-- fecha_hora_retiro: timestamp (nullable = true)
 |-- anio_retiro: integer (nullable = true)
 |-- mes_retiro: integer (nullable = true)
 |-- dia_retiro: integer (nullable = true)
 |-- minuto_retiro: integer (nullable = true)
 |-- segundo_retiro: integer (nullable = true)



## Añadir columnas para Fecha y Hora de arribo

In [7]:
from pyspark.sql import functions as F

# Crear la columna fecha_hora_arribo
df_Arr = df.withColumn("fecha_hora_arribo", F.concat_ws(" ", "Fecha_Arribo", F.date_format("Hora_Arribo", "HH:mm:ss"))) \
           .withColumn("fecha_hora_arribo", F.to_timestamp("fecha_hora_arribo"))

# Extraer año, mes, día, minuto y segundo para fecha_hora_arribo
df_Arr = df_Arr.withColumn("anio_arribo", F.year("fecha_hora_arribo")) \
               .withColumn("mes_arribo", F.month("fecha_hora_arribo")) \
               .withColumn("dia_arribo", F.dayofmonth("fecha_hora_arribo")) \
               .withColumn("minuto_arribo", F.minute("fecha_hora_arribo")) \
               .withColumn("segundo_arribo", F.second("fecha_hora_arribo"))

# Mostrar el DataFrame resultante
df_Arr.show(10)


+--------------+------------+----+---------------------+------------+--------------------+---------------------+------------+--------------------+-------------------+-----------+----------+----------+-------------+--------------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|         Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|         Hora_Arribo|  fecha_hora_arribo|anio_arribo|mes_arribo|dia_arribo|minuto_arribo|segundo_arribo|
+--------------+------------+----+---------------------+------------+--------------------+---------------------+------------+--------------------+-------------------+-----------+----------+----------+-------------+--------------+
|             M|          28|2401|                  118|  2012-12-01|2024-10-20 00:00:...|                  118|  2012-12-01| 2024-10-20 00:15:00|2012-12-01 00:15:00|       2012|        12|         1|           15|             0|
|             M|          26|1540|                   73|  2012-12-01|2024-10-20 

In [8]:
df_Arr.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: integer (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- Fecha_Retiro: date (nullable = true)
 |-- Hora_Retiro: timestamp (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- Fecha_Arribo: date (nullable = true)
 |-- Hora_Arribo: timestamp (nullable = true)
 |-- fecha_hora_arribo: timestamp (nullable = true)
 |-- anio_arribo: integer (nullable = true)
 |-- mes_arribo: integer (nullable = true)
 |-- dia_arribo: integer (nullable = true)
 |-- minuto_arribo: integer (nullable = true)
 |-- segundo_arribo: integer (nullable = true)



### Muestra de sólo las columnas que se acaban de añadir

In [9]:
# Seleccionar las columnas de arribo y retiro
df_combinado = df_Arr.join(
    df_Ret,
    on="Bici",  
    how="inner"  
).select(
    "fecha_hora_arribo",
    "anio_arribo",
    "mes_arribo",
    "dia_arribo",
    "minuto_arribo",
    "segundo_arribo",
    "fecha_hora_retiro",
    "anio_retiro",
    "mes_retiro",
    "dia_retiro",
    "minuto_retiro",
    "segundo_retiro"
)

# Mostrar el DataFrame combinado
df_combinado.show()


+-------------------+-----------+----------+----------+-------------+--------------+-------------------+-----------+----------+----------+-------------+--------------+
|  fecha_hora_arribo|anio_arribo|mes_arribo|dia_arribo|minuto_arribo|segundo_arribo|  fecha_hora_retiro|anio_retiro|mes_retiro|dia_retiro|minuto_retiro|segundo_retiro|
+-------------------+-----------+----------+----------+-------------+--------------+-------------------+-----------+----------+----------+-------------+--------------+
|2012-12-01 13:58:40|       2012|        12|         1|           58|            40|2012-12-01 13:43:43|       2012|        12|         1|           43|            43|
|2012-12-01 13:58:40|       2012|        12|         1|           58|            40|2012-12-02 07:33:39|       2012|        12|         2|           33|            39|
|2012-12-01 13:58:40|       2012|        12|         1|           58|            40|2012-12-03 08:50:18|       2012|        12|         3|           50|        

## Crear tablas temporales

In [10]:
df_combinado = df_combinado.select(
    "fecha_hora_arribo",
    "anio_arribo",
    "mes_arribo",
    "dia_arribo",
    "minuto_arribo",
    "segundo_arribo",
    "fecha_hora_retiro",
    "anio_retiro",
    "mes_retiro",
    "dia_retiro",
    "minuto_retiro",
    "segundo_retiro"
)
df_temp = df.select(
    "Genero_Usuario",
    "Edad_Usuario",
    "Bici",
    "Ciclo_Estacion_Retiro"
)
df_temp = df_temp.join(df_combinado)

df_temp.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: integer (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- fecha_hora_arribo: timestamp (nullable = true)
 |-- anio_arribo: integer (nullable = true)
 |-- mes_arribo: integer (nullable = true)
 |-- dia_arribo: integer (nullable = true)
 |-- minuto_arribo: integer (nullable = true)
 |-- segundo_arribo: integer (nullable = true)
 |-- fecha_hora_retiro: timestamp (nullable = true)
 |-- anio_retiro: integer (nullable = true)
 |-- mes_retiro: integer (nullable = true)
 |-- dia_retiro: integer (nullable = true)
 |-- minuto_retiro: integer (nullable = true)
 |-- segundo_retiro: integer (nullable = true)



In [11]:
df_temp.createOrReplaceTempView("Ecobicitemporal")

24/10/20 12:58:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
